In [1]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql import types as T
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

spark = SparkSession.builder \
    .appName("Medallion Architecture") \
    .getOrCreate()

24/05/23 18:06:27 WARN Utils: Your hostname, Aymans-MacBook-Pro-2.local resolves to a loopback address: 127.0.0.1; using 192.168.50.146 instead (on interface en4)
24/05/23 18:06:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/23 18:06:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
raw_path = os.path.join("..","data","*.csv")

df = spark.read.csv(raw_path, header=True, inferSchema=True)
df.printSchema()
df.show(5)

root
 |-- YEAR: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUC: integer (nullable = true)
 |-- ETHNIC: integer (nullable = true)
 |-- RACE: integer (nullable = true)
 |-- GENDER: integer (nullable = true)
 |-- SPHSERVICE: integer (nullable = true)
 |-- CMPSERVICE: integer (nullable = true)
 |-- OPISERVICE: integer (nullable = true)
 |-- RTCSERVICE: integer (nullable = true)
 |-- IJSSERVICE: integer (nullable = true)
 |-- MH1: integer (nullable = true)
 |-- MH2: integer (nullable = true)
 |-- MH3: integer (nullable = true)
 |-- SUB: integer (nullable = true)
 |-- MARSTAT: integer (nullable = true)
 |-- SMISED: integer (nullable = true)
 |-- SAP: integer (nullable = true)
 |-- EMPLOY: integer (nullable = true)
 |-- DETNLF: integer (nullable = true)
 |-- VETERAN: integer (nullable = true)
 |-- LIVARAG: integer (nullable = true)
 |-- NUMMHS: integer (nullable = true)
 |-- TRAUSTREFLG: integer (nullable = true)
 |-- ANXIETYFLG: integer (nullable = true)
 |-- ADHDFLG

24/05/23 18:06:45 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+
|YEAR|AGE|EDUC|ETHNIC|RACE|GENDER|SPHSERVICE|CMPSERVICE|OPISERVICE|RTCSERVICE|IJSSERVICE|MH1|MH2|MH3|SUB|MARSTAT|SMISED|SAP|EMPLOY|DETNLF|VETERAN|LIVARAG|NUMMHS|TRAUSTREFLG|ANXIETYFLG|ADHDFLG|CONDUCTFLG|DELIRDEMFLG|BIPOLARFLG|DEPRESSFLG|ODDFLG|PDDFLG|PERSONFLG|SCHIZOFLG|ALCSUBFLG|OTHERDISFLG|STATEFIP|DIVISION|REGION|     CASEID|
+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+
|2021| 

In [3]:
df.show()
df.describe().show()

+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+
|YEAR|AGE|EDUC|ETHNIC|RACE|GENDER|SPHSERVICE|CMPSERVICE|OPISERVICE|RTCSERVICE|IJSSERVICE|MH1|MH2|MH3|SUB|MARSTAT|SMISED|SAP|EMPLOY|DETNLF|VETERAN|LIVARAG|NUMMHS|TRAUSTREFLG|ANXIETYFLG|ADHDFLG|CONDUCTFLG|DELIRDEMFLG|BIPOLARFLG|DEPRESSFLG|ODDFLG|PDDFLG|PERSONFLG|SCHIZOFLG|ALCSUBFLG|OTHERDISFLG|STATEFIP|DIVISION|REGION|     CASEID|
+----+---+----+------+----+------+----------+----------+----------+----------+----------+---+---+---+---+-------+------+---+------+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+
|2021| 

+-------+--------------------+------------------+-------------------+-----------------+-----------------+------------------+-------------------+-------------------+-------------------+-------------------+-------------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-------------------+------------------+------------------+------------------+------------------+------------------+-------------------+-------------------+-------------------+-------------------+--------------------+-------------------+-------------------+--------------------+--------------------+--------------------+-------------------+-------------------+-------------------+------------------+------------------+------------------+--------------------+
|summary|                YEAR|               AGE|               EDUC|           ETHNIC|             RACE|            GENDER|         SPHSERVICE|         CMPSERVICE|         OPISERVICE|         RTCSERVICE|   

Bronze Layer
Staging (Bronze) Layer:
- i. Create a bronze table to store the raw data from the CSV file.
- ii. Perform basic data validation and cleaning, such as handling null values, data types, and column names.
- iii. Partition the bronze table by an appropriate column(s) to improve query performance.
- iv. Prepare to discuss how you would handle logging and exception handling to capture any issues during the bronze layer processing

In [4]:
bronze_path = os.path.join("..","data","medallion_layers","bronze")
df.write.partitionBy("GENDER", "RACE", "ETHNIC", "MARSTAT", "EMPLOY").mode("overwrite").parquet(bronze_path)



24/05/23 18:07:34 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828 bytes) of heap memory
Scaling row group sizes to 88.13% for 8 writers
24/05/23 18:07:35 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828 bytes) of heap memory
Scaling row group sizes to 88.13% for 8 writers
24/05/23 18:07:40 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828 bytes) of heap memory
Scaling row group sizes to 88.13% for 8 writers
24/05/23 18:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828 bytes) of heap memory
Scaling row group sizes to 88.13% for 8 writers
24/05/23 18:07:45 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828 bytes) of heap memory
Scaling row group sizes to 88.13% for 8 writers
24/05/23 18:07:47 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828 bytes) of heap memory
Scaling row group sizes to 88.13% for 8 writers
24/05/23 18:07:49 WARN MemoryManager: Total allocation exceeds 95.00% (946,339,828

In [5]:
bronze_cleaned_path = os.path.join("..","data","medallion_layers","bronze_cleaned")
bronze_df = spark.read.parquet(bronze_path, header=True, inferSchema=True)
cleaned_bronze_df = bronze_df.na.drop()
cleaned_bronze_df.show(5)
cleaned_bronze_df.printSchema()

+----+---+----+----------+----------+----------+----------+----------+---+---+---+---+------+---+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+------+----+------+-------+------+
|YEAR|AGE|EDUC|SPHSERVICE|CMPSERVICE|OPISERVICE|RTCSERVICE|IJSSERVICE|MH1|MH2|MH3|SUB|SMISED|SAP|DETNLF|VETERAN|LIVARAG|NUMMHS|TRAUSTREFLG|ANXIETYFLG|ADHDFLG|CONDUCTFLG|DELIRDEMFLG|BIPOLARFLG|DEPRESSFLG|ODDFLG|PDDFLG|PERSONFLG|SCHIZOFLG|ALCSUBFLG|OTHERDISFLG|STATEFIP|DIVISION|REGION|     CASEID|GENDER|RACE|ETHNIC|MARSTAT|EMPLOY|
+----+---+----+----------+----------+----------+----------+----------+---+---+---+---+------+---+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+------+----+------+-------+------+
|2021| 

In [6]:
cleaned_bronze_df.write.partitionBy("GENDER", "RACE", "ETHNIC", "MARSTAT", "EMPLOY").mode("overwrite").parquet(bronze_cleaned_path)

Silver Layer
- i. Create a silver table to store the transformed and enriched data.
- ii. Perform more complex transformations (See Transformation Exercise
for Senior Data Engineer for details), such as data aggregations, and data quality checks.
- iii. Partition the silver table by an appropriate column(s) to improve query performance.
- iv. Implement schema checks to ensure the transformed data matches the expected schema.
- v. Prepare to discuss how you would handle monitoring to track the success and failure of the silver layer processing.

In [7]:

bronze_cleaned_path = os.path.join("..","data","medallion_layers","bronze_cleaned")

cleaned_bronze_df = spark.read.parquet(bronze_cleaned_path)

cleaned_bronze_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUC: integer (nullable = true)
 |-- SPHSERVICE: integer (nullable = true)
 |-- CMPSERVICE: integer (nullable = true)
 |-- OPISERVICE: integer (nullable = true)
 |-- RTCSERVICE: integer (nullable = true)
 |-- IJSSERVICE: integer (nullable = true)
 |-- MH1: integer (nullable = true)
 |-- MH2: integer (nullable = true)
 |-- MH3: integer (nullable = true)
 |-- SUB: integer (nullable = true)
 |-- SMISED: integer (nullable = true)
 |-- SAP: integer (nullable = true)
 |-- DETNLF: integer (nullable = true)
 |-- VETERAN: integer (nullable = true)
 |-- LIVARAG: integer (nullable = true)
 |-- NUMMHS: integer (nullable = true)
 |-- TRAUSTREFLG: integer (nullable = true)
 |-- ANXIETYFLG: integer (nullable = true)
 |-- ADHDFLG: integer (nullable = true)
 |-- CONDUCTFLG: integer (nullable = true)
 |-- DELIRDEMFLG: integer (nullable = true)
 |-- BIPOLARFLG: integer (nullable = true)
 |-- DEPRESSFLG: integer (nullable =

1. Data Type Conversion
    - a. Convert the categorical variables (GENDER, RACE, ETHNICITY, MARITAL, EMPLOY, INCOME) from numeric values to appropriate string or categorical data types.
    
    - b. Ensure that the CASEID variable is stored as an integer data type.
    
    - c. Convert the numeric variables (MENHLTH, PHYHLTH, POORHLTH) to float data types.
    
    - d. Validate the data types for all variables to ensure they are consistent with the expected formats.
    
    - e. Provide a summary of the data type conversions performed.

In [8]:
gender = {
    1: "Male",
    2: "Female",
    -9: "Missing/unknown/not collected/invalid"
}

race = {
    1: "American Indian/Alaska Native",
    2: "Asian",
    3: "Black or African American",
    4: "Native Hawaiian or Other Pacific Islander",
    5: "White",
    6: "Some other race alone/two or more races",
    -9: "Missing/unknown/not collected/invalid"
}

marital_status = {
    1: "Never married",
    2: "Now married",
    3: "Separated",
    4: "Divorced, widowed",
    -9: "Missing/unknown/not collected/invalid"
}

employment_status = {
    1: "Full-time",
    2: "Part-time",
    3: "Employed full-time/part-time not differentiated",
    4: "Unemployed",
    5: "Not in labor force",
    -9: "Missing/unknown/not collected/invalid"
}

ethnicity = {
    1: "Mexican",
    2: "Puerto Rican",
    3: "Other Hispanic or Latino origin",
    4: "Not of Hispanic or Latino origin",
    -9: "Missing/unknown/not collected/invalid"
}

bronze_cleaned_path = os.path.join("..","data","medallion_layers","bronze_cleaned")

cleaned_bronze_df = spark.read.parquet(bronze_cleaned_path)

cleaned_bronze_df.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUC: integer (nullable = true)
 |-- SPHSERVICE: integer (nullable = true)
 |-- CMPSERVICE: integer (nullable = true)
 |-- OPISERVICE: integer (nullable = true)
 |-- RTCSERVICE: integer (nullable = true)
 |-- IJSSERVICE: integer (nullable = true)
 |-- MH1: integer (nullable = true)
 |-- MH2: integer (nullable = true)
 |-- MH3: integer (nullable = true)
 |-- SUB: integer (nullable = true)
 |-- SMISED: integer (nullable = true)
 |-- SAP: integer (nullable = true)
 |-- DETNLF: integer (nullable = true)
 |-- VETERAN: integer (nullable = true)
 |-- LIVARAG: integer (nullable = true)
 |-- NUMMHS: integer (nullable = true)
 |-- TRAUSTREFLG: integer (nullable = true)
 |-- ANXIETYFLG: integer (nullable = true)
 |-- ADHDFLG: integer (nullable = true)
 |-- CONDUCTFLG: integer (nullable = true)
 |-- DELIRDEMFLG: integer (nullable = true)
 |-- BIPOLARFLG: integer (nullable = true)
 |-- DEPRESSFLG: integer (nullable =

In [12]:
# cleaned_bronze_df.withColumn()

In [17]:
def create_map_udf(mapping_dict):
    return F.udf(lambda key: mapping_dict.get(key, "Unknown"), T.StringType())

race_map_udf = create_map_udf(race)
gender_map_udf = create_map_udf(gender)
marital_status_map_udf = create_map_udf(marital_status)
employment_status_map_udf = create_map_udf(employment_status)
ethnicity_map_udf = create_map_udf(ethnicity)


silver_df = cleaned_bronze_df
silver_df = silver_df.withColumn("GENDER", gender_map_udf("GENDER"))
silver_df = silver_df.withColumn("RACE", race_map_udf("RACE"))
silver_df = silver_df.withColumn("MARSTAT", marital_status_map_udf("MARSTAT"))
silver_df = silver_df.withColumn("EMPLOY", employment_status_map_udf("EMPLOY"))
silver_df = silver_df.withColumn("ETHNIC", ethnicity_map_udf("ETHNIC"))

# Ensure CASEID is stored as an integer
silver_df = silver_df.withColumn("CASEID", F.col("CASEID").cast("integer"))

# Convert numeric variables to float
# numeric_columns = ["MENHLTH", "PHYHLTH", "POORHLTH"]
# numeric_columns = ["MH1","MH2","MH3"]
# for column in numeric_columns:
#     silver_df = silver_df.withColumn(column, F.col(column).cast("float"))

# Validate the data types for all variables
silver_df.printSchema()

# Summary of data type conversions
data_type_summary = {
    "GENDER": "Converted to string/categorical using predefined mapping",
    "RACE": "Converted to string/categorical using predefined mapping",
    "ETHNIC": "Converted to string/categorical using predefined mapping",
    "MARSTAT": "Converted to string/categorical using predefined mapping",
    "EMPLOY": "Converted to string/categorical using predefined mapping",
    "CASEID": "Converted to integer",
    "MENHLTH": "Converted to float",
    "PHYHLTH": "Converted to float",
    "POORHLTH": "Converted to float"
}

# Print summary
for column, summary in data_type_summary.items():
    print(f"{column}: {summary}")

root
 |-- YEAR: integer (nullable = true)
 |-- AGE: integer (nullable = true)
 |-- EDUC: integer (nullable = true)
 |-- SPHSERVICE: integer (nullable = true)
 |-- CMPSERVICE: integer (nullable = true)
 |-- OPISERVICE: integer (nullable = true)
 |-- RTCSERVICE: integer (nullable = true)
 |-- IJSSERVICE: integer (nullable = true)
 |-- MH1: integer (nullable = true)
 |-- MH2: integer (nullable = true)
 |-- MH3: integer (nullable = true)
 |-- SUB: integer (nullable = true)
 |-- SMISED: integer (nullable = true)
 |-- SAP: integer (nullable = true)
 |-- DETNLF: integer (nullable = true)
 |-- VETERAN: integer (nullable = true)
 |-- LIVARAG: integer (nullable = true)
 |-- NUMMHS: integer (nullable = true)
 |-- TRAUSTREFLG: integer (nullable = true)
 |-- ANXIETYFLG: integer (nullable = true)
 |-- ADHDFLG: integer (nullable = true)
 |-- CONDUCTFLG: integer (nullable = true)
 |-- DELIRDEMFLG: integer (nullable = true)
 |-- BIPOLARFLG: integer (nullable = true)
 |-- DEPRESSFLG: integer (nullable =

In [18]:
silver_df.show()

+----+---+----+----------+----------+----------+----------+----------+---+---+---+---+------+---+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+--------+------+-----------+------+-----+--------------------+-------------+--------------------+
|YEAR|AGE|EDUC|SPHSERVICE|CMPSERVICE|OPISERVICE|RTCSERVICE|IJSSERVICE|MH1|MH2|MH3|SUB|SMISED|SAP|DETNLF|VETERAN|LIVARAG|NUMMHS|TRAUSTREFLG|ANXIETYFLG|ADHDFLG|CONDUCTFLG|DELIRDEMFLG|BIPOLARFLG|DEPRESSFLG|ODDFLG|PDDFLG|PERSONFLG|SCHIZOFLG|ALCSUBFLG|OTHERDISFLG|STATEFIP|DIVISION|REGION|     CASEID|GENDER| RACE|              ETHNIC|      MARSTAT|              EMPLOY|
+----+---+----+----------+----------+----------+----------+----------+---+---+---+---+------+---+------+-------+-------+------+-----------+----------+-------+----------+-----------+----------+----------+------+------+---------+---------+---------+-----------+--------+

2. Data Normalization and Standardization
- a. Normalize the numeric variables (MENHLTH, PHYHLTH, POORHLTH) using min-max scaling to bring them to a common scale between 0 and 1.
- b. Standardize the numeric variables (MENHLTH, PHYHLTH, POORHLTH) using z-score normalization to have a mean of 0 and a standard deviation of 1.
- c. Ensure that the normalized and standardized variables are stored in new columns, keeping the original variables intact.
- d. Provide a summary of the normalization and standardization techniques used and the rationale behind them.

3. Data Partitioning and Sampling
- a. Split the dataset into training and testing sets, ensuring a representative split based on the demographic variables (GENDER, RACE, ETHNICITY, MARITAL, EMPLOY, INCOME).
- b. Implement stratified sampling to create the training and testing sets, maintaining the same proportions of the demographic variables in both sets.
- c. Create a validation set from the training set using a similar stratified sampling approach.
- d. Ensure that the dataset splits are stored in separate files or data structures, with appropriate naming conventions and metadata.
- e. Provide a detailed report on the data partitioning and sampling process, including the rationale for the chosen techniques and the characteristics of the resulting datasets.

Business (Gold) Layer
- i. Create a gold table to store the final, curated data.
- ii. Perform additional transformations and calculations to create
business-ready datasets.
- iii. Partition the gold table by an appropriate column(s) to improve query performance.
- iv. Prepare to discuss how you would handle schema checks to ensure the
final data meets the expected schema.
- v. Prepare to discuss how you would handle monitoring to track the success
and failure of the gold layer processing